# Démonstration 10

## 1. Algorithmes de points intérieurs

In [ ]:
using LinearAlgebra
using JuMP, HiGHS

In [ ]:
# Klee-Minty problem

n = 3

m = Model(HiGHS.Optimizer)

@variable(m, x[1:n] >= 0)
for i in 1:n
   @constraint(m, 2*sum(10^(i-j)*x[j] for j = 1:i-1)+x[i] <= 100^(i-1))
end
@objective(m, Max, sum(10^(n-j)*x[j] for j = 1:n))

print(m)

In [ ]:
status = optimize!(m)

In [ ]:
value.(x)

In [ ]:
c = [[-10^(n-i) for i = 1:n]; [0 for i = 1:n]]
b = [100^(i-1) for i = 1:n]
A = [1 [0*i for i in 1:n-1]']
for i = 2:n
    A = [A; [[2*10^(i-j) for j in 1:i-1]' 1 [0 for i = 1:n-i]']]
end
B = A
A = [A I]
x = [0.5]
for i = 2:n
    x = [ x ; (0.5*b[i]-dot(A[i,1:i-1],x[1:i-1]))]
end
x = [ x; b-B*x ]
A*x

In [ ]:
x

In [ ]:
m, n = size(A)
y = [-1 for i = 1:m]
s = c-A'y

In [ ]:
function primaldual_path_following(c,A,b,x,y,s, μ = 100, α = 0.1, γ=0.1, epsilon=1e-9, max_iter=1000)

    for ν = 1:10

        iter = max_iter

        for k = 1:max_iter
            
            X = diagm(x)
            S = diagm(s)
        H = [ zeros(n,n) A' I ;
              A zeros(m,m) zeros(m,n);
              S zeros(n,m) X ]
        t = [ zeros(n) ; zeros(m); μ*ones(n)-X*S*ones(n) ]
        Δ = H\t
        α = 0.01
        new = [x; y; s]+α*Δ
        x = new[1:n]
        y = new[n+1:n+m]
        s = new[n+m+1:2*n+m]
        if (norm(x.*s-μ*ones(n)) <= 1/10^(ν/2))
            iter = k
            break
        end

        end

        println(ν, ". ", iter, " ", x.*s)
        μ = γ*μ

    end
    
    return x
end

In [ ]:
primaldual_path_following(c,A,b,x,y,s)

On converge, mais l'implémentation n'est pas très efficace.

## Question 2

Nous considérons un problème de programmation linéaire sous forme standard et
nous supposons que les ensemble réalisables du primal et du dual ont des
intérieurs non vides. Montrer que si l'ensemble réalisable du primal est borné,
alors l'ensemble réalisable du dual ne peut pas être borné. Est-il possible
d'avoir des ensembles réalisables non bornés pour le primal et le dual?

Soit $\left(x(\mu), y(\mu), s(\mu) \right)$, $0<\mu<\infty$. Si l'ensemble réalisable de primal
est borné, $x(\mu)$ converge vers le centre analytique de l'ensemble réalisable
du primal. Dès lors, $x(\mu)$ tend vers un certain $x^{\star} $ pour $\mu \to
\infty$, avec $x^{\star} $ fini.  D'autre part, on sait que

\begin{align*}
c^{t} x(\mu) - b^{t} y(\mu) = \eta \mu
\end{align*}
Dès lors, pour $\mu \to \infty$, 
\begin{align*}
\lim_{\mu\to \infty} b^t y(\mu) = c^t x^{\star} - \infty = - \infty
\end{align*}
Dès lors, $y(\mu)$ ne peut pas converger vers un certain $y^{\star} $ fini comme $\mu$ tend vers l'infini. 
Or ce serait le cas si l'ensemble dual était borné.

On peut avoir des ensembles non bornés pour le primal et le dual. Considérions la paire
\begin{align*}
\min\ & x_1 + x_2 \\
\text{s.à. } & x_1 = 1,\\
& x_1, x_2 \ge 0.
\end{align*}
\begin{align*}
\max\ & \lambda \\
\text{s.à. } & \lambda \le 1.
\end{align*}
Les deux ensembles ont des intérieurs non vides et ne sont pas bornés.

# Question 3

Considérons le polyhèdre $P = \{ x \,|\, Ax \leq b \}$. Supposons que $P$ est borné et que son intérieur est non-vide, ou en d'autres termes, qu'il existe $x$ tel que $Ax < b$.

- Écrivez la fonction potentiel à minimiser pour trouver le centre analytique de $P$.
- En annulant le gradient de la fonction potentiel, écrivez le système non-linéaire à résoudre pour trouver le centre analytique.
- Le système obtenu au point précédent peut être résolu à l'aide de la méthode de Newton. Écrivez une itération de la résolution du système pas la méthode de Newton.

Notons $a_i^T$ la $i^e$ ligne de $A$. La contrainte $Ax \leq b$ peut se réécrire
$$
b - a_i^Tx \geq 0,\ i = 1,\ldots,m,
$$
en supposant que $A \in R^{m \times n}$.

La fonction potentiel à minimiser s'écrit dès lors
$$
\Psi(x) = -\sum_{i = 1}^m \log(b_i-a_i^Tx)
$$
Son gradient s'écrit
$$
\nabla_x \Psi(x) =
\begin{pmatrix}
 \sum_{i = 1}^m \frac{a_{i1}}{b_i-a_i^Tx} \\
 \sum_{i = 1}^m \frac{a_{i2}}{b_i-a_i^Tx} \\
\vdots \\
 \sum_{i = 1}^m \frac{a_{in}}{b_i-a_i^Tx}
\end{pmatrix}
=
 \sum_{i = 1}^m \frac{a_{i}}{b_i-a_i^Tx} \\
= A^T\begin{pmatrix}
\frac{1}{b-a_1x} \\
\frac{1}{b-a_2x} \\
\vdots \\
\frac{1}{b-a_mx}
\end{pmatrix}
$$
que nous cherchons à annuler.
En posant
$$
s_i = b_i - a_i^Tx, \quad y_i = \frac{1}{s_i},\ i = 1,\ldots,m,
$$
le système peut se réécrire comme
$$
A^Ty = 0,\ \textrm{avec}\ y = \begin{pmatrix} y_1 \\ \vdots \\ y_m \end{pmatrix}.
$$

Pour obtenir l'itération de Newton, nous devons tout d'abord écrire la matrice hessienne de la fonction potentiel:
$$
\nabla^2_{xx} \Psi(x) =
\sum_{i = 1}^m \frac{a_{i}a_i^T}{(b_i-a_i^Tx)^2} = A^TS^{-2}A = A^TY^2A,
$$
Une itération de Newton consiste à prendre
$$
x^+ = x - (\nabla^2_{xx} \Psi(x))^{-1}\nabla_x \Psi(x) = x - (A^TY^2A)^{-1}A^Ty.
$$

# Implémentation de la Méthode de Mehrotra

Arnaud L'Heureux, Décembre 2019

Étant donné la paire primale-duale d'un problème de programmation linéaire : 
$$\min_x c^Tx$$
$$\text{s.à.}\ \ Ax = b$$
$$x \geq 0$$
et
$$\max_{\lambda,s} b^T\lambda$$
$$\text{s.à.}\ \ A^T\lambda + s = c$$
$$s \geq 0$$
Nous avons les conditons KKT suivantes qui doivent être satisfaites pour que la solution au problème soit optimale:
$$Ax = b$$
$$A^T\lambda + s = c$$
$$XSe = 0$$
$$(x,s) \geq 0$$
où $X$ et $S$ sont des matrices diagonales ayant $x$ et $s$ comme diagonales. La condition $Ax = b$, nous indique que la solution est réalisable pour le primal alors que $A^T\lambda + s = c$ implique que le dual est réalisable. Tout point réalisable pour le primal ainsi que le dual satisfaisant la condition $XSe = 0$ implique que ce point est optimal pour le primal et le dual.

In [ ]:
using LinearAlgebra

# Heuristique proposée par Mehrotra
function genStart(A:: Matrix, b:: Vector, c:: Vector)
    x̅ = A'*((A*A')\b)
    λ̅ = (A*A')\A*c
    s̅ = c - A'*λ̅

    δx = max(-1.5*minimum(x̅),0)
    δs = max(-1.5*minimum(s̅),0)

    # δx et δs sont des scalaires, et doivent être ajoutés à chaque composante, c'est pourquoi nous utilisons .+
    num = dot(x̅ .+ δx, s̅ .+ δs)
    δx̅ = δx + num/(2*sum(s̅ .+ δs))
    δs̅ = δs + num/(2*sum(x̅ .+ δx))

    return (x̅ .+ δx̅, λ̅ , s̅ .+ δs̅)
end

L'algorithme de Mehrotra suit.

In [ ]:
using SparseArrays

function Mehrotra(A:: Matrix, b:: Vector, c:: Vector,
                  ε:: Float64, maxIter:: Int64 = 1000, sparseMode:: Bool = True)

    (m,n) = size(A)  # calcul de la dimension de A
    
    if (sparseMode)
        temp = sparse(A)
        density = nnz(temp)/(m*n)
       
        # Si la densité de A est inférieure à 20%, nous travaillons avec une matrice creuse
        if (density < .2)
            A = sparse(A)
            b = sparse(b)
            c = sparse(c)
        end
    end

    # Calcul du point de départ
    res = genStart(A,b,c)
    x = res[1]
    λ = res[2]
    s = res[3]
    
    iter = 0

    # Nous utilisons le critère d'arrêt par défaut dans le test de boucle.
    # Le test de convergence se fera en interne, et nous pourrons alors quitter la fonction.
    while (iter < maxIter)
        iter += 1
        
        # Calcul des résidus
        rp = A*x-b
        rd = A'*λ + s - c
        rc = x .* s

        # μ est aligné sur le saut de dualité
        μ = dot(x,s)/n
        
        # Nous testons la qualité de la solution
        if (max(μ, norm(rd), norm(rp)) < ε)
            return (x,λ,s)
        end
        
        # Calcul de M = AXS^-1A' et du factory de Cholesky R de M (M=R'R)
        M = A*diagm(x./s)*A'
        R = cholesky(M).U

        # Calcul des termes de droite
        td = rp - A*((rc - x.*rd)./s)
        
        # Étape prédictrice
        Δλ_p = R\(R'\td)
        Δs_p = rd - A'*Δλ_p
        Δx_p = (rc - x.*Δs_p)./s
 
        # Calculs des longueur de pas potentiels les plus longs
        # Pas primal
        α_p = 1/maximum([1 ; Δx_p ./ x])
        # Pas dual
        α_d = 1/maximum([1 ; Δs_p ./ s])
        
        # Paramètre centralisant
        σ = (dot(x - α_p*Δx_p, s - α_d*Δs_p)/(n*μ))^3
        
        # Étape correctrice
        rc = rc .- σ*μ + Δx_p.*Δs_p
        td = rp - A*((rc - x.*rd)./s)
        
        Δλ_c = R\(R'\td)
        Δs_c = rd - A'*Δλ_c
        Δx_c = (rc - x.*Δs_c)./s
        
        η = max(0.995, 1-μ)
        
        # Pas primal
        α_p = η/maximum([η ; Δx_c./x])
        # Pas dual
        α_d = η/maximum([η ; Δs_c./s])
        
        # Mise à jour da la solution
        x -= α_p*Δx_c
        λ -= α_d*Δλ_c
        s -= α_d*Δs_c
    end
end

### Exemple

Soit le problème:
\begin{align*}
\min z\ &= 5x_1 + 2x_2 -4x_3 \\
\text{s.à. } & 6x_1 + x_2 - 2x_3 - x_4 = 5 \\
& x_1 + x_2 + x_3 + x_5 = 4 \\
& 6x_1 + 4x_2 -2x_3 - x_6= 10 \\
& x_i \geq 0, \ i = 1,\ldots,6.
\end{align*}

In [ ]:
A = [6 1 -2 -1 0 0; 1 1 1 0 1 0; 6 4 -2 0 0 -1]
b = [5; 4; 10]
c = [5; 2; -4; 0; 0; 0]

Nous commençons par trouver un point de départ $(x_0, \lambda_0, s_0)$.

In [ ]:
start = genStart(A,b,c)

println(A*start[1]-b)
println(A'*start[2]-c)
println(start[1], " ", start[3])

Nous voyons que seules les contraintes de non-négativité sont satisfaites, mais ni $x_0$, ni $\lambda_0$, ne sont réalisables.

Appliquons l'algorithe de Mehrotra.

In [ ]:
x = Mehrotra(A,b,c,1e-10,10,true)[1]

println("x = ", x)
println("Ax = ", A*x)
println("z = c^Tx = ", dot(c,x))

Nous obtenons la solution:

$$x = \begin{pmatrix}
  0.9999999999999999 \\
  1.6666666666666656 \\
  1.3333333333333355 \\
  3.5867130386728067 \times 10^{-19}\\
  1.5140739781564760 \times 10^{-19}\\
  3.8753111845137296 \times 10^{-19}
 \end{pmatrix}  \approx \begin{pmatrix}
  1 \\
  \frac{5}{3} \\
  \frac{4}{3} \\
  0\\
  0\\
  0
 \end{pmatrix}
$$
La solution est
$$x_1 = 1, \ \ x_2 = \frac{5}{3}, \ \ x_3 = \frac{4}{3}, \ \ x_4 = x_5 = x_6 = 0,$$
qui est bien réalisable puisque $Ax = b$. Cette solution nous donne la valeur de la fonction objectif $z = 3$.

# Programmation linéaire en nombres entiers

In [ ]:
using LinearAlgebra
using JuMP, Gurobi

Les logiciels commerciaux supportent, le plus souvent très efficacement, la programmation linéaire mixte en nombre entiers. Considérons par exemple le problème ci-dessous.

In [ ]:
m = Model(Gurobi.Optimizer)

# Variables
@variable(m, 0<= x1 <=10)
@variable(m, x2 >=0, Int)
@variable(m, x3, Bin)

# Fonction objectif
@objective(m, Max, x1 + 2x2 + 5x3)

# Contraintes
@constraint(m, constraint1, -x1 +  x2 + 3x3 <= -5)
@constraint(m, constraint2,  x1 + 3x2 - 7x3 <= 10)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2), " x3 = ", value(x3))

`HiGHS` n'est néanmoins pas en reste.

In [ ]:
set_optimizer(m, HiGHS.Optimizer)

In [ ]:
status = optimize!(m)

println("Optimal Solutions:")
println("x1 = ", value(x1), " x2 = ", value(x2), " x3 = ", value(x3))

## Branch & Bound

Soit le problème de programmation linéaire en nombres entiers suivant
\begin{align*}
\min\ z &= -10x_1 -20x_2 \\
\text{s.à } 
& 5x_1 + 8 x_2 \le 60 ,\\
&x_1 \le 8\\
&x_2 \le 4\\
& x_1,x_2 \ge 0 \text{ et entiers.}
\end{align*}

Nous allons procéder par résolution graphique. Il est évidemment possible d'utiliser d'autres outils, comme des solveurs de programmation linéaire.

In [ ]:
using Plots

## Itération 1

Nous commençons en résolvant la relaxation linéaire
\begin{align*}
\min\ z\ &= -10x_1 -20x_2 \\
\text{s.à } &
5x_1 + 8 x_2 \le 60\\
&x_1 \le 8\\
&x_2 \le 4\\
& x_1,x_2 \ge 0.
\end{align*}

In [ ]:
x1=0:0.01:12
g2 = (x1 -> 7-x1/2 ≥ 0 ? 15/2-5/8*x1 : NaN)
g3 = (x1 -> 0 )
g3 = (x1 -> 4 )

objOpt = (x1 -> -1/5*x1+(282.5)/50 ≥ 0 ? -1/2*x1+(136)/20 : NaN)

pyplot()

plot()

plot!(x1,g2,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :red,
         style=:auto,
         color=:red,
         label="5x1 + 8x2 = 60")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=0")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=4")

plot!(x1,objOpt, color=:black, linewidth=2, linestyle=:solid, label="-10*x1-20*x2=-136")


scatter!([5.6],[4],label="Optimum", color=:red)
vline!([0], label="x1 = 0", color=:orange)
vline!([8], label="x1 = 6", color=:yellow)

plot!(                                       
    #size=(800, 600),                                                                             
    xticks = 0:12,   
    yticks = 0:8,                          
                                                  
    ylabel = "x2", 
    xlabel = "x1",                       
    
    legend=:bottomleft,
    title  = "Programme linéaire (P)"                    
    )

Solution : $x_1 = 5.6$; $x_2 = 4$; $\bar{z} = -136$ 

## Itération 2

Nous branchons sur $x_1$ pour créer deux problèmes, le premier avec $x_1 \leq 5$, l'autre avec $x_1 \geq 6$.

Commençons par le premier problème
\begin{align*}
\min\ z\ &= -10x_1 -20x_2 \\
\text{s.à } &
5x_1 + 8 x_2 \le 60\\
&x_1 \le 5\\
&x_2 \le 4\\
& x_1, x_2 \ge 0.
\end{align*}

In [ ]:
x1=0:0.01:12
g2 = (x1 -> 7-x1/2 ≥ 0 ? 15/2-5/8*x1 : NaN)
g3 = (x1 -> 0 )
g3 = (x1 -> 4 )

objOpt = (x1 -> -1/5*x1+(282.5)/50 ≥ 0 ? -1/2*x1+(130)/20 : NaN)

pyplot()

plot()

plot!(x1,g2,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :red,
         style=:auto,
         color=:red,
         label="5x1 + 8x2 = 60")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=0")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=4")

plot!(x1,objOpt, color=:black, linewidth=2, linestyle=:solid, label="-10*x1-20*x2=-130")


scatter!([5],[4],label="Optimum", color=:red)
vline!([0], label="x1 = 0", color=:orange)
vline!([5], label="x1 = 6", color=:yellow)

plot!(                                       
    #size=(800, 600),                                                                             
    xticks = 0:12,   
    yticks = 0:8,                          
                                                  
    ylabel = "x2", 
    xlabel = "x1",                       
    
    legend=:bottomleft,
    title  = "Programme linéaire (P)"                    
    )

Solution : $x_1 = 5$; $x_2 = 4$; $\bar{z} = -130$

La solution est entière, aussi nous ne devrons plus brancher à partir de ce sous-prblème, et nous avons une borne supérieure sur la valeur optimale.

## Itération 3

Considérons maintenant le problème
\begin{align*}
\min\ z\ &= -10x_1 -20x_2 \\
\text{s.à } &
5x_1 + 8 x_2 \le 60\\
&x_1 \le 8\\
&x_1 \ge 6\\
&x_2 \le 4\\
& x_1, x_2 \ge 0.
\end{align*}

In [ ]:
x1=6:0.01:12
g2 = (x1 -> 7-x1/2 ≥ 0 ? 15/2-5/8*x1 : NaN)
g3 = (x1 -> 0 )
g3 = (x1 -> 4 )


objOpt = (x1 -> -1/5*x1+(282.5)/50 ≥ 0 ? -1/2*x1+(135)/20 : NaN)

pyplot()

plot()

plot!(x1,g2,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :red,
         style=:auto,
         color=:red,
         label="5x1 + 8x2 = 60")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=0")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=4")

plot!(x1,objOpt, color=:black, linewidth=2, linestyle=:solid, label="-10*x1-20*x2=-135")


scatter!([6],[3.75],label="Optimum", color=:red)
vline!([0], label="x1 = 0", color=:orange)
vline!([6], label="x1 = 6", color=:yellow)
vline!([8], label="x1 = 6", color=:yellow)

plot!(                                       
    #size=(800, 600),                                                                             
    xticks = 0:12,   
    yticks = 0:8,                          
                                                  
    ylabel = "x2", 
    xlabel = "x1",                       
    
    legend=:bottomleft,
    title  = "Programme linéaire (P)"                    
    )

Solution : $x_1 = 6$; $x_2 = 3,75$; $\bar{z} = -135$

La solution n'est pas entière, et nous avons une borne inférieure sur la solution optimale au niveau des descendants du noeud courant. Comme cette borne est inférieure à la borne supérieure trouvée prédémment, il est possible de trouver une meilleure solution au problème initial en branchant à partir de ce noeud.

## Itération 4

Nous branchons à présent sur $x_2$, en considérant $x_2 \leq 3$ et $x_2 \geq 3$.

Le premier problème est
\begin{align*}
\min\ z\ &= -10x_1 -20x_2 \\
\text{s.à } &
5x_1 + 8 x_2 \le 60\\
&x_1 \le 8\\
&x_1 \ge 6\\
&x_2 \le 3\\
& x_1, x_2 \ge 0.
\end{align*}

In [ ]:
x1=6:0.01:12
g2 = (x1 -> 7-x1/2 ≥ 0 ? 15/2-5/8*x1 : NaN)
g3 = (x1 -> 0 )
g3 = (x1 -> 3 )


objOpt = (x1 -> -1/5*x1+(282.5)/50 ≥ 0 ? -1/2*x1+(132)/20 : NaN)

pyplot()

plot()

plot!(x1,g2,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :red,
         style=:auto,
         color=:red,
         label="5x1 + 8x2 = 60")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=0")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=4")

plot!(x1,objOpt, color=:black, linewidth=2, linestyle=:solid, label="-10*x1-20*x2=-132")


scatter!([7.2],[3],label="Optimum", color=:red)
vline!([0], label="x1 = 0", color=:orange)
vline!([6], label="x1 = 6", color=:yellow)
vline!([8], label="x1 = 6", color=:yellow)

plot!(                                       
    #size=(800, 600),                                                                             
    xticks = 0:12,   
    yticks = 0:8,                          
                                                  
    ylabel = "x2", 
    xlabel = "x1",                       
    
    legend=:bottomleft,
    title  = "Programme linéaire (P)"                    
    )

Solution : $x_1 = 7,2$; $x_2 = 3$; $\bar{z} = -132$

La solution n'est pas entière et la borne inférieure sur la valeur optimale à partir de ce noeud est -132, qui est plus basse que la meilleure borne supérieure connue, aussi nous ne pouvons pas élager le noeud à ce stade.

## Itération 5

Ce problème implique dès lors la création de deux nouveaux sous-problème, en branchant à nouveau sur $x_1$.

Considérons tout d'abord
\begin{align*}
\min\ z\ &= -10x_1 -20x_2 \\
\text{s.à } &
5x_1 + 8 x_2 \le 60\\
&x_1 \le 7\\
&x_1 \ge 6\\
&x_2 \le 3\\
& x_1, x_2 \ge 0.
\end{align*}

In [ ]:
x1=6:0.01:12
g2 = (x1 -> 7-x1/2 ≥ 0 ? 15/2-5/8*x1 : NaN)
g3 = (x1 -> 0 )
g3 = (x1 -> 3 )


objOpt = (x1 -> -1/5*x1+(282.5)/50 ≥ 0 ? -1/2*x1+(130)/20 : NaN)

pyplot()

plot()

plot!(x1,g2,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :red,
         style=:auto,
         color=:red,
         label="5x1 + 8x2 = 60")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=0")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=4")

plot!(x1,objOpt, color=:black, linewidth=2, linestyle=:solid, label="-10*x1-20*x2=-130")


scatter!([7],[3],label="Optimum", color=:red)
vline!([0], label="x1 = 0", color=:orange)
vline!([6], label="x1 = 6", color=:yellow)
vline!([7], label="x1 = 6", color=:yellow)

plot!(                                       
    #size=(800, 600),                                                                             
    xticks = 0:12,   
    yticks = 0:8,                          
                                                  
    ylabel = "x2", 
    xlabel = "x1",                       
    
    legend=:bottomleft,
    title  = "Programme linéaire (P)"                    
    )

Solution : $x_1 = 7$; $x_2 = 3$; $\bar{z} = -130$

La solution est entière, pour une valeur objectif de -130, égale à la meilleure valeur actuellement connue.

## Itération 6

Le problème est maintenant
\begin{align*}
\min\ z\ &= -10x_1 -20x_2 \\
\text{s.à } &
5x_1 + 8 x_2 \le 60\\
&x_1 \ge 7\\
&x_1 \le 8\\
&x_2 \le 3\\
& x_1, x_2 \ge 0.
\end{align*}

In [ ]:
x1=7:0.01:12
g2 = (x1 -> 7-x1/2 ≥ 0 ? 15/2-5/8*x1 : NaN)
g3 = (x1 -> 0 )
g3 = (x1 -> 3 )


objOpt = (x1 -> -1/5*x1+(282.5)/50 ≥ 0 ? -1/2*x1+(130)/20 : NaN)

pyplot()

plot()

plot!(x1,g2,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :red,
         style=:auto,
         color=:red,
         label="5x1 + 8x2 = 60")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=0")
plot!(x1, g3,fillrange = 0,
         fillalpha = 0.3,
         fillcolor = :green,
         color=:green,
         label="x2=4")

plot!(x1,objOpt, color=:black, linewidth=2, linestyle=:solid, label="-10*x1-20*x2=-130")


scatter!([7],[3],label="Optimum", color=:red)
vline!([0], label="x1 = 0", color=:orange)
vline!([7], label="x1 = 7", color=:yellow)
vline!([8], label="x1 = 8", color=:yellow)

plot!(                                       
    #size=(800, 600),                                                                             
    xticks = 0:12,   
    yticks = 0:8,                          
                                                  
    ylabel = "x2", 
    xlabel = "x1",                       
    
    legend=:bottomleft,
    title  = "Programme linéaire (P)"                    
    )

Solution :  𝑥1=8 ;  𝑥2=2,5 ;  𝑧¯=−130

La valeur n'est pas entière, mais la valeur de la fonction objectif associée à la solution du problème relaxé n'est pas meilleure que la meilleure valeur connue, aussi nous ne pourrons pas l'améliorer en branchant à partir du noeud actuel, aussi élaguons-nous le noeud.

## Itération 7

Nous devons revenir au noeud non-élagué avec la contrainte $x_2 \geq 4$.

En considérant les deux contraintes $x_2 \leq 4$ et $x_2 \geq 4$, nous voyons que $x_2$ doit être fixé à 4, et le problème se réécrit 
\begin{align*}
\min\ z\ &= -10x_1 - 80 \\
\text{s.à } & 5x_1 \le 28 \\
&x_1 \le 8\\
&x_1 \ge 6\\
& x_1 \ge 0.
\end{align*}
Mais $x_1$ ne peut être à la fois plus grand que 6 et plus petit que 28/5, aussi le problème est non-réalisable, et nous pouvons élaguer ce noeud.

Il n'y a plus de noeud à élaguer aussi non pouvons conclure. La valeur optimale est -130, atteinte en (5,4) et (7,3).